In [2]:
import pandas as pd
import numpy as np

# Генерация примерных данных
np.random.seed(42)  # Для воспроизводимости

# Создание столбцов данных
n = 1000  # Количество строк
data = pd.DataFrame({
    'Дата': pd.date_range(start='2024-01-01', periods=n, freq='D'),
    'Расстояние': np.random.uniform(1, 100, n),  # Расстояние от 1 до 100 км
    'Объем груза': np.random.uniform(0.1, 10, n),  # Объем от 0.1 до 10 м³
    'Стоимость заказа': np.random.uniform(50, 1000, n) + 
                        np.random.normal(0, 10, n)  # Добавим шум
})

# Проверка первых строк
print(data.head())


        Дата  Расстояние  Объем груза  Стоимость заказа
0 2024-01-01   38.079472     1.932816        309.219763
1 2024-01-02   95.120716     5.464819        290.799919
2 2024-01-03   73.467400     8.742164        917.777545
3 2024-01-04   60.267190     7.349026        273.409333
4 2024-01-05   16.445845     8.084955        320.471680


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Создание scaler
scaler = StandardScaler()

# Масштабируем признаки
data[['Расстояние', 'Объем груза']] = scaler.fit_transform(data[['Расстояние', 'Объем груза']])

# Целевая переменная и признаки
X = data[['Расстояние', 'Объем груза']]
y = data['Стоимость заказа']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
pip install xgboost


   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 217.9 kB/s eta 0:09:34
   ---------------------------------------- 0.0/124.9 MB 217.9 kB/s eta 0:09:34
   ---------------------------------------- 0.0/124.9 MB 245.8 kB/s eta 0:08:29
   ---------------------------------------- 0.1/124.9 MB 438.1 kB/s eta 0:04:45
   ---------------------------------------- 0.2/124.9 MB 655.4 kB/s eta 0:03:11
   ---------------------------------------- 0.3/124.9 MB 874.6 kB/s eta 0:02:23
   ---------------------------------------- 0.4/124.9 MB 1.2 MB/s eta 0:01:43
   ---------------------------------------- 0.5/124.9 MB 1.4 MB/s eta 0:01:28
   ---------------------------------------- 0.7/124.9 MB 1.7 MB/s eta 0:01:15
   ---------------------------------------- 0.9/124.9 MB 1.8 MB/s eta 0:01:09
   ---------------------------------------- 1.3/124.9 MB 2.5 MB/s 

In [7]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

# Линейная регрессия
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Градиентный бустинг
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [8]:
from sklearn.metrics import mean_squared_error, r2_score

# Предсказания
y_pred_linear = linear_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

# Метрики
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"Linear Regression - MSE: {mse_linear}, R2: {r2_linear}")
print(f"XGBoost - MSE: {mse_xgb}, R2: {r2_xgb}")


Linear Regression - MSE: 79625.80292137405, R2: -0.007968826294834308
XGBoost - MSE: 91313.20471144588, R2: -0.15591755940095386


In [9]:
xgb_model_small = XGBRegressor(n_estimators=50, max_depth=3, learning_rate=0.1, random_state=42)
xgb_model_small.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [10]:
# Пример взвешивания редких событий
weights = np.where(X_train['Расстояние'] > X_train['Расстояние'].quantile(0.9), 2, 1)

xgb_model_weighted = XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
xgb_model_weighted.fit(X_train, y_train, sample_weight=weights)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)